In [1]:
periodo = 'pre-pandemia'

# Importe as bibliotecas necessárias

In [2]:
%matplotlib inline
from IPython.display import clear_output
import tqdm
import os
import pandas as pd
import numpy as np
import seaborn as sns
import random
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree._classes import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.neural_network._multilayer_perceptron import MLPClassifier
from sklearn.linear_model._logistic import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from yellowbrick.style.colors import resolve_colors
from sklearn.model_selection._validation import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from unidecode import unidecode
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.decomposition import PCA
import pickle
#import statsmodels.api as sm
import platform 
np.random.seed(42)

In [3]:
def strfdelta(tdelta, fmt="{hours}:{minutes}:{seconds}"):
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

# Carregando Base de dados

In [4]:
df = pd.read_csv(f'twitterbase_{periodo}.csv', sep=';')
df.head()

,volumeTweets_media,volumeTweets_variancia,volumeTweets_mediaMovelPonterada,volumeTweets_entropia,indiceInsonia_media,indiceInsonia_variancia,indiceInsonia_mediaMovelPonterada,indiceInsonia_entropia,pronome1Pessoa_media,pronome1Pessoa_variancia,...,links_entropia,midia_media,midia_variancia,midia_mediaMovelPonterada,midia_entropia,curtidas_media,curtidas_variancia,curtidas_mediaMovelPonterada,curtidas_entropia,classe
0,0.232915,0.315346,1.605634,0.593198,0.086239,0.111151,0.578638,0.307958,0.016736,0.022035,...,0.324054,0.006974,0.006925,0.049296,0.041577,0.103208,0.179027,0.725352,0.320571,depressao
1,2.510294,47.111659,17.270431,1.838695,0.620732,1.595054,4.288012,1.377158,0.483824,3.058562,...,0.159815,0.000000,0.000000,0.000000,0.000000,1.048529,5.416763,7.237741,1.303106,depressao
2,0.356322,1.985678,2.425234,0.612006,0.146022,0.323983,1.003827,0.443252,0.112644,0.251679,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.002299,0.002294,0.002336,0.016263,controle
3,0.154891,0.891770,1.027701,0.334143,0.034081,0.039944,0.243190,0.160835,0.008152,0.013520,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,1.033288,0.891967,0.183438,controle
4,30.116279,1931.916712,206.861111,2.842062,2.232019,15.639582,16.762029,2.242101,10.953488,269.858302,...,2.348284,0.441860,1.176852,3.388889,0.795581,4.023256,75.557599,25.500000,1.857342,depressao


# Codificando a variável categórica

In [5]:
label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

# Preparando os dados

In [6]:
def train_val_test(X, y, train_ratio, val_ratio, test_ratio):
    assert sum([train_ratio, test_ratio, val_ratio])==1.0, "wrong given ratio, all ratios have to sum to 1.0"
    assert X.shape[0]==len(y), "X and y shape mismatch"

    ind_train = int(round(X.shape[0]*train_ratio))
    ind_test = int(round(X.shape[0]*(train_ratio+test_ratio)))

    X_train = X[:ind_train]
    X_test = X[ind_train:ind_test]
    X_val = X[ind_test:]

    y_train = y[:ind_train]
    y_test = y[ind_train:ind_test]
    y_val = y[ind_test:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [7]:


def avaliacao(classificador, predicao, x, y, nome_classificador):
    '''
    Computa as mericas de avaliação dos clasficadores
    '''
        
    precisao = precision_score(y, predicao)
    revocacao = recall_score(y, predicao)
    f1score = f1_score(y, predicao)
    acuracia = accuracy_score(y, predicao)
    return [precisao, revocacao, f1score, acuracia]
    
def modeloML(model, x, x_train, y_train, x_val, y_val, nome_classificador, atributo, i):
    clear_output()
    print(f'Experimento {i}')
    print(nome_classificador)
    
    model.fit(x_train, y_train)
    predicao = cross_val_predict(model, x_val, y_val, cv=cv)
    #predicao = GridSearchCV(model, {}, cv=cv, verbose=10, n_jobs=3).fit(x_train, y_train).predict(x_val)
    #predicao = cross_val_predict(GridSearchCV(model, {}, cv=cv, verbose=10).fit(x_train, y_train).best_estimator_, x_val, y_val, cv=cv)
    resultados = avaliacao(model, predicao, x_val, y_val, nome_classificador)
    df_resultados.loc[nome_classificador][f'exp_basico_DeChoudhury_precisao'] = resultados[0]
    df_resultados.loc[nome_classificador][f'exp_basico_DeChoudhury_recall'] = resultados[1]
    df_resultados.loc[nome_classificador][f'exp_basico_DeChoudhury_f1_score'] = resultados[2]
    df_resultados.loc[nome_classificador][f'exp_basico_DeChoudhury_acuracia'] = resultados[3]
    
    #plotarMatrizConfusao(y_val, predicao, nome_classificador)

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [9]:
if not os.path.exists(f'df_resultados_{periodo}_f1Score.xlsx'):
    metricas = ['acuracia', 'precisao', 'recall', 'f1_score']
    colunas = [f'exp_basico_DeChoudhury_{i}' for i in metricas]
    modelos = ['Regressão Logistica (Baseline)', 'Análise Discriminante Linear', 'KNN', 'Naive Bayes',
               'Árvore de Decisão', 'Floresta Randômica', 'Gradient Boosting', 'Perceptron Multicamadas', 'SVM',
               'Bagging', 'Boosting', 'Votação Hard', 'Votação Soft'
              ]
    df_resultados = pd.DataFrame(columns=colunas, index=modelos)
    df_resultados.index.name = "Modelo"
else:
    df_resultados = pd.read_excel(f'df_resultados_{periodo}_f1Score.xlsx', index_col=0)
    
    

In [10]:
df_resultados

,exp_basico_DeChoudhury_acuracia,exp_basico_DeChoudhury_precisao,exp_basico_DeChoudhury_recall,exp_basico_DeChoudhury_f1_score
Modelo,,,,
Regressão Logistica (Baseline),NaN,NaN,NaN,NaN
Análise Discriminante Linear,NaN,NaN,NaN,NaN
KNN,NaN,NaN,NaN,NaN
Naive Bayes,NaN,NaN,NaN,NaN
Árvore de Decisão,NaN,NaN,NaN,NaN
Floresta Randômica,NaN,NaN,NaN,NaN
Gradient Boosting,NaN,NaN,NaN,NaN
Perceptron Multicamadas,NaN,NaN,NaN,NaN
SVM,NaN,NaN,NaN,NaN


In [11]:
def executar(x, y, atributo, i):

    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test(x, y, train_ratio, validation_ratio, test_ratio)

    baseline = LogisticRegression(random_state=42, C=1e-06, n_jobs=-1)
    analiseDiscriminanteLinear = LinearDiscriminantAnalysis(shrinkage=0, solver='eigen', tol=1e-6)
    knn = KNeighborsClassifier(n_neighbors=9, n_jobs=-1)
    naiveBayes = BernoulliNB(alpha=1e-09)

    arvoreDecisao = DecisionTreeClassifier(random_state=42, max_depth=9)
    florestaRandomica = RandomForestClassifier(random_state=42, max_depth=10, n_estimators=90, verbose=10, n_jobs=-1)
    gradientBoosting = GradientBoostingClassifier(random_state=42, n_estimators=100, verbose=10)
    perceptronMulticamadas = MLPClassifier(random_state=42, hidden_layer_sizes=(10,30,10), max_iter=100, verbose=10)
    svm = LinearSVC(random_state=42, C=1, max_iter=1000, verbose=10)
    
    modeloML(baseline, x, x_train, y_train, x_val, y_val, 'Regressão Logistica (Baseline)', atributo, i)
    modeloML(analiseDiscriminanteLinear, x, x_train, y_train, x_val, y_val, 'Análise Discriminante Linear', atributo, i)
    modeloML(naiveBayes, x, x_train, y_train, x_val, y_val, 'Naive Bayes', atributo, i)
    modeloML(arvoreDecisao, x, x_train, y_train, x_val, y_val, 'Árvore de Decisão', atributo, i)
    modeloML(knn, x, x_train, y_train, x_val, y_val, 'KNN', atributo, i)
    modeloML(svm, x, x_train, y_train, x_val, y_val, 'SVM', atributo, i)
    modeloML(florestaRandomica, x, x_train, y_train, x_val, y_val, 'Floresta Randômica', atributo, i)
    modeloML(perceptronMulticamadas, x, x_train, y_train, x_val, y_val, 'Perceptron Multicamadas', atributo, i)
    modeloML(gradientBoosting, x, x_train, y_train, x_val, y_val, 'Gradient Boosting', atributo, i)
    
    florestaRandomica.set_params(verbose=0)
    gradientBoosting.set_params(verbose=0)
    perceptronMulticamadas.set_params(verbose=0)
    svm.set_params(verbose=0)

    listaEstimators = [('Análise Discriminante Linear', analiseDiscriminanteLinear), 
                       ('Naive Bayes', naiveBayes), 
                       ('KNN', knn), 
                       ('Árvore de Decisão', arvoreDecisao), 
                       ('Floresta Randômica', florestaRandomica), 
                       ('Perceptron Multicamadas', perceptronMulticamadas),
                       ('Gradient Boosting', gradientBoosting),
                       ('SVM', svm),
                      ]


    bagging = BaggingClassifier(base_estimator=perceptronMulticamadas, n_estimators=10, random_state=42, verbose=10, n_jobs=-1)
    boosting = AdaBoostClassifier(base_estimator=florestaRandomica, n_estimators=10, random_state=42) 
    votingHard = VotingClassifier(voting='hard', estimators=listaEstimators, verbose=10, n_jobs=-1)
    votingSoft = VotingClassifier(voting='soft', estimators=listaEstimators[:-1], verbose=10, n_jobs=-1)

    modeloML(votingHard, x, x_train, y_train, x_val, y_val, 'Votação Hard', atributo, i)
    modeloML(votingSoft, x, x_train, y_train, x_val, y_val, 'Votação Soft', atributo, i)
    modeloML(bagging, x, x_train, y_train, x_val, y_val, 'Bagging', atributo, i)
    modeloML(boosting, x, x_train, y_train, x_val, y_val, 'Boosting', atributo, i)

In [12]:
choq = ['volumeTweets_media', 'volumeTweets_mediaMovelPonterada', 
        'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada',
        'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 
        'pronome2Pessoa_mediaMovelPonterada', 'pronome2Pessoa_entropia',
        'pronome3Pessoa_media', 'pronome3Pessoa_mediaMovelPonterada',
        'valencia_mediaMovelPonterada', 'valencia_entropia', 
        'ativacao_mediaMovelPonterada', 'ativacao_entropia', 
        'termosDepressivos_variancia', 'termosDepressivos_mediaMovelPonterada',
        'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada',
        'medicamentosAntiDepressivo_media', 'medicamentosAntiDepressivo_mediaMovelPonterada']
caracteresOrientais = ['caracteresOrientais_variancia', 'caracteresOrientais_mediaMovelPonterada']
emojis = ['emojis_variancia', 'emojis_entropia']
links = ['links_mediaMovelPonterada', 'links_entropia']
midia = ['midia_variancia', 'midia_mediaMovelPonterada']
curtidas = ['curtidas_media', 'curtidas_mediaMovelPonterada']

In [13]:
exp = choq
x = df[exp].copy()
y = df['classe'].copy()
x_new = zscore.fit_transform(x)
executar(x_new, y, '', 1)

Experimento 1
Boosting


In [14]:
df_resultados_round = df_resultados.sort_values(by='exp_basico_DeChoudhury_f1_score',ascending=False).astype('float64').round(3) * 100
df_resultados_round.style.set_precision(1)

,exp_basico_DeChoudhury_acuracia,exp_basico_DeChoudhury_precisao,exp_basico_DeChoudhury_recall,exp_basico_DeChoudhury_f1_score
Modelo,,,,
Gradient Boosting,77.2,75.5,90.1,82.2
Boosting,77.3,76.2,88.8,82.0
Floresta Randômica,76.6,74.8,90.1,81.8
Bagging,76.5,75.9,87.6,81.3
Perceptron Multicamadas,76.1,75.5,87.4,81.0
Votação Soft,76.1,75.8,86.7,80.9
Votação Hard,75.7,75.7,85.8,80.5
Árvore de Decisão,74.7,74.1,86.9,80.0
KNN,70.5,70.6,84.7,77.0


In [15]:
df_resultados_round.to_excel(f'df_resultados_{periodo}_f1Score.xlsx', encoding='utf-8')